# Gale Shapley Matching with Bandits

To decide a matching between $\mathcal{B}$ and $\mathcal{L}$, we introduce the binary decision variable $\mathbf{x}$ := $(x_{bl})_{(b, l)\in \mathcal{B} \times \mathcal{L}}$ such that $x_{bl}$ = 1 if the loan from lender $l$ is assigned and accepted by borrower $b$ and 0 otherwise. It is supposed that when the borrower $b$ and the lender $l$ are matched, the borrower and the lender gain utilities of $u_b(h)$ and $u_l(h)$ respectively. To simplify  our settings, we assume that all utilities $u_b(h)$ and $u_l(h)$ are non-negative, and that for a lender $l$, $u_b(l)$ $\neq$ $u_{b'}(l)$, for $b \neq b'$ and similarly, for a borrower $b$, $u_l(b)$ $\neq$ $u_{l'}(b)$, for $l \neq l'$. The borrower-lender pair $(b, l)$ yields the utility of $u_{bl}:= u_b + u_l $ and the total utility of a matching $\mathbf{x} \in {0, 1}^{|\mathcal{B}| \times |\mathcal{L}|}$ is given by

\begin{equation*}
    \sum_{b \in \mathcal{B}} \sum_{l \in \mathcal{L}} u_{bl} x_{bl}
\end{equation*}
In our work, we consider a \textcolor{red}{many-many matching} where each borrower is matched to many lenders and each lender is also matched to multiple borrowers. For a matching $\mathbf{x}$, let $m_x(b)\in \mathcal{L}$ be the lender to which the borrower $b$ is assigned, and $M_x(l) \subseteq \mathcal{B}$ be the set of borrowers that are assigned to lender $l$, that is,
\begin{align*}
    m_x(b) = l \Longleftrightarrow x_{bl} = 1 \\
    M_x(l) := \{l \in \mathcal{L} \ | \ x_{bl} = 1\}
\end{align*}
\\

\noindent \textbf{Definition 1} Let $\mathbf{x} \in \{0, 1\}^{|\mathcal{B}| \times |\mathcal{L}|}$ be a matching. A pair $(b, l) \in \mathcal{B} \times \mathcal{L}$ is a blocking pair for $\mathbf{x}$ if the following conditions are satisfied:

\begin{enumerate}
    \item $x_{bl}=0$
    \item $x(b)$ is null or $u_b(l)$ $>$ $u_{b}(m_x(b))$
    \item $|M_x(b)|$ $< q_b$ or $\exists$ $l' \in M_x(b)$ and such that $u_b(l) > u_b(l')$.
\end{enumerate} 
\vspace*{.1in}

Since the individual rationality condition is fulfilled from Assumption 1 for all $(b, l) \in \mathcal{B} \times \mathcal{L}$, a stable matching is defined based on the absence of blocking pairs as follows: a matching $\mathbf{x}$ is stable if it does not have any blocking pairs. \\

\noindent \textbf{Definition 2.} A matching $\mathbf{x}$ is stable if it does not have blocking pairs. The stable matching problem can be characterized by the following linear inequalities as mentioned in \cite{baiou2000stable} and is formalized using the following inequality:

\begin{equation} \label{eq:eq1}
    q_b x_{bl} + q_b\sum_{l' \succ_b l }x_{bl'} + \sum_{b' \succ_{l} b} x_{b'l} \geq q_l
\end{equation}
The proof of the statement can be found in the same paper \cite{baiou2000stable}. We use this notation to arrive at a linear program that maximizes the utility of the matching while minimizing the number of blocking pairs. Defining a binary decision variable $\mathbf{w}$ := $(w_{b, l})$ $\in \{0, 1\}^{|\mathcal{B}| \times |\mathcal{L}|}$, we use the following constraint: 

\begin{equation}
        q_b x_{bl} + q_b\sum_{l' \succ_b l }x_{bl'} + \sum_{b' \succ_{l} b} x_{b'l} \geq q_l (1- w_{bl})
\end{equation}

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
import random
from collections import defaultdict 
import math

In [2]:
def model_gs_matching(n_b, n_l, u_b, u_l, c, q, obj_util, lambda_1, lambda_2, LogToConsole=True, TimeLimit=60):
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = {}
    w = {}
    for b_idx in range(1, n_b+1):
        x[b_idx] = {}
        w[b_idx] = {}
        for l_idx in range(1, n_l+1): 
            x_name = "x_{}_{}".format(b_idx, l_idx)
            x[b_idx][l_idx] = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY, name=x_name)
            w_name = "w_{}_{}".format(b_idx, l_idx)
            w[b_idx][l_idx] = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY, name=w_name)

    for l_idx in range(1, n_l+1):
        model.addConstr(quicksum(x[b_idx][l_idx] for b_idx in range(1, n_b+1)) <= 1)
    
    for b_idx in range(1, n_b+1):
        model.addConstr(quicksum((q[l_idx]*x[b_idx][l_idx]) for l_idx in range(1, n_l+1)) >= c[b_idx])
    
    for b_ix in range(1, n_b+1):
        for l_idx in range(1, n_l+1):
            constr_obj_1 = c[b_idx]*x[b_idx][l_idx]
            constr_obj_2 = 0
            constr_obj_3 = 0
            
            for b_idx_2 in range(1, n_b+1):
                if b_idx != b_idx_2:
#                     print(b_idx, b_idx_2)
                    if u_l[l_idx][b_idx] < u_l[l_idx][b_idx_2]:
                        constr_obj_2 += (x[b_idx][l_idx])
            constr_obj_2 *= c[b_idx] 

            for l_idx_2 in range(1, n_l+1):
                if l_idx != l_idx_2:
                    if u_b[b_idx][l_idx] < u_b[b_idx][l_idx_2]:
                        constr_obj_3 += (q[l_idx] * x[b_idx][l_idx])
                        
            model.addConstr((constr_obj_1 + constr_obj_2 + constr_obj_3) >= (c[b_idx]* (1-w[b_idx][l_idx])))
    
    model.setObjective(lambda_1*quicksum(obj_util[b_idx][l_idx]*x[b_idx][l_idx] for l_idx in range(1, n_l+1) for b_idx in range(1, n_b+1)) - \
                       lambda_2*quicksum(w[b_idx][l_idx] for l_idx in range(1, n_l+1) for b_idx in range(1, n_b+1)) , GRB.MAXIMIZE)
    model.optimize()
    if model.status != 2:
        print("Optimal Solution not found !!!")
        return -1, -1
    
    borrower_matches = {}
    lender_matches = {}
    for b_idx in range(1, n_b+1):
        borrower_matches[b_idx] = []
#         print("Borrower {} matched to lenders: ".format(b_idx))
        for l_idx in range(1, n_l+1):
            if x[b_idx][l_idx].X == 1:
#                 print(l_idx)
                borrower_matches[b_idx].append(l_idx)
                if l_idx not in lender_matches:
                    lender_matches[l_idx] = -1
                lender_matches[l_idx] = b_idx
    
    return borrower_matches, lender_matches, model.objVal

In [3]:
def lender_utility(l, b, sim_values, amount_lenders, borrower_rates):
    return sim_values[b][l] #+ borrower_rates[b]*amount_lenders[l]

# Risk preference not considered for now
def borrower_utility(l, b, sim_values, risk_preference):
    return (sim_values[b][l] + risk_preference[b][l])/2.0 # 2.0 is for normalization

In [4]:

VARIANCE = 0.3
NUM_SIMS_PER_STEP=50
INFTY = 10

def rewards(mean, variance):
    return np.random.normal(mean, variance, 1)[0]

def average_reward(base_reward, rewards_list):
    if len(rewards_list) == 0:
        return base_reward
    else:
        return (base_reward + np.sum(rewards_list))/(len(rewards_list)+1)

def reward_ucb(base_reward, rewards_list, time): 
    if len(rewards_list) == 0:
        return average_reward(base_reward, rewards_list) + INFTY
    else:
        return average_reward(base_reward, rewards_list) + np.sqrt(1.5*np.log(time+1)/len(rewards_list))
       
def cb_arms(base_reward, rewards_list, time):    
    if len(rewards_list) > 0:
        return average_reward(base_reward, rewards_list) - np.sqrt(1.5*np.log(time+1)/len(rewards_list)), \
                average_reward(base_reward, rewards_list) + np.sqrt(1.5*np.log(time+1)/len(rewards_list))
    else:
        return average_reward(base_reward, rewards_list) - INFTY, average_reward(base_reward, rewards_list) + INFTY
    
def gs_bandit_method_baseline(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, preference_borrowers, preference_lenders):
    u = defaultdict(lambda: defaultdict(float))
    obj = defaultdict(lambda: defaultdict(float))
    for l_idx in range(1, n_l+1):
        for b_idx in range(1, n_b+1):
            # we optimize for lender_borrower returns
            u[l_idx][b_idx] = u_b[b_idx][l_idx] + u_l[l_idx][b_idx] # lender utility + borrower utility
            obj[b_idx][l_idx] = u[l_idx][b_idx] # objective fn utility same as lender-borrower utility

    borrower_matches_optimal, lender_matches_optimal, objVal = model_gs_matching(n_b, n_l, u_b, u, c, q, obj, lambda_1, lambda_2, LogToConsole=False)
  
    return lender_matches_optimal, objVal
            

def find_interesction_arms(lcb_m, ucb_m, interval_list_arms, b):
    intersecting_arms = []
    for b_idx in interval_list_arms:
        if b == b_idx:
            continue
        lcb, ucb = interval_list_arms[b_idx]
        if lcb < ucb_m or ucb > lcb_m:
            intersecting_arms.append(l_idx)
    
    return intersecting_arms


In [34]:
    
    
def gs_bandit_method_basic(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, preference_borrowers, preference_lenders):
    T = 20 # horizon, n
    
    rewards_list_l = defaultdict(lambda: defaultdict(list))
    util_send_l = defaultdict(lambda: defaultdict(float))
    
    # For each lender l, initialize the current utility for a borrower b as u_l(b)
    # and then initilize the util_send list for matching with the current utility
    for l_idx in range(1, n_l+1):
        for b_idx in range(1, n_b+1):
            util_send_l[l_idx][b_idx] =  u_l[l_idx][b_idx]
    
    # Baseline GS
    lender_matches_optimal = []
    while ( len(lender_matches_optimal) < n_l):
        lender_matches_optimal, objVal_optimal = gs_bandit_method_baseline(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, preference_borrowers, preference_lenders)
    
    
    regret_lender_t = defaultdict(lambda: defaultdict(list))
    
    for s_idx in range(NUM_SIMS_PER_STEP):
        print("Simulation no.: " + str(s_idx))
        for t in range(1, T+1):
            print("Matching time step " + str(t))
    #         u = {}
    #         for b_idx in range(1, n_b+1):
    #             u[b_idx] = {}
    #             for l_idx in range(1, n_l+1):
    #                 u[b_idx][l_idx] = u_l[l_idx][b_idx] #lender optimal
            borrower_matches, lender_matches, objVal = model_gs_matching(n_b, n_l, u_b, util_send_l, c, q, util_send_l, lambda_1, lambda_2, LogToConsole=False)
#             if borrower_matches == -1 or lender_matches == -1:
#                 print("Non-optimal lending")
#                 for l_idx in range(1, n_l+1):
#                     regret_lender_t[l_idx].append(-1)
#                 continue

            for l_idx in range(1, n_l+1):
                if l_idx in lender_matches:
                    b_match = lender_matches[l_idx] # matched borrower
                    rewards_list_l[l_idx][b_match].append(rewards(u_b[b_match][l_idx], VARIANCE)) # update the reward list for l-b pair
                    # For each lender l, the current utility for a borrower b becomes u_l(b) + [N(1, 0.25) + \sum (rewards_list)]/(1 + len(rewards_list))
                    # util_send becomes curr_util[l][b] + sqrt(2*log t/T_{b,l}(t-1))
#                     curr_util_l[l_idx][b_match] = average_reward(u_l[l_idx][b_match], rewards_list_l[l_idx][b_match])
                    util_send_l[l_idx][b_match] = reward_ucb(u_l[l_idx][b_match], rewards_list_l[l_idx][b_match], t)

                    # print(lender_matches_optimal[l_idx], b_match)
                    r = u_l[l_idx][lender_matches_optimal[l_idx]] - u_l[l_idx][b_match]
                else:
#                     if len(regret_lender_t[l_idx][s_idx]) > 0:
#                         r = regret_lender_t[l_idx][s_idx][-1] # same as before
#                     else:
                    r =   regret_lender_t[l_idx][t-1][s_idx]
                    
              
                regret_lender_t[l_idx][t].append(r)
            
#                 print("Lender {} with regret {}".format(l_idx, r))
    
    return regret_lender_t

In [27]:
def gs_bandit_phases(n_b, n_l, preference_borrowers, preference_lenders):
    T = 10 # horizon, n
    
    curr_util_l = defaultdict(lambda: defaultdict(list))
    rewards_list_l = defaultdict(lambda: defaultdict(list))
    util_send_l = defaultdict(lambda: defaultdict(float))
    
    # For each lender l, initialize the current utility for a borrower b as u_l(b) + N(1, 0.25)
    # and then initilize the util_send list for matching with the current utility
    for l_idx in range(1, n_l+1):
        for b_idx in range(1, n_b+1):
            curr_util_l[l_idx][b_idx] = average_reward(u_l[l_idx][b_idx], rewards_list_l[l_idx][b_idx])
            util_send_l[l_idx][b_idx] = curr_util_l[l_idx][b_idx]
    
    # Baseline GS
    lender_matches_optimal = []
    while ( len(lender_matches_optimal) < n_l):
        lender_matches_optimal, objVal_optimal = gs_bandit_method_baseline(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, preference_borrowers, preference_lenders)
    
    
    regret_lender_t = defaultdict(lambda: defaultdict(list))
    
    for s_idx in range(NUM_SIMS_PER_STEP):
        print("Simulation no.: " + str(s_idx))
        for t in range(1, T+1):
            print("Matching time step " + str(t))
            lenders_unmatched = list(range(n_l))
            # Proceed in phases
            
            cb_arms = defaultdict(lambda:defaultdict(int))
            
            # Initialize the lcb and the ucb for all the arms for each lender
            for l_idx in range(1, n_l+1):
                for b_idx in range(1, n_b+1):
                    cb_arms[l_idx][b_idx] = cb_arms(u_l[l_idx][bidx], [], 0)
            
            
            while (len(lenders_unmatched) > 0):
                borrower_matches, lender_matches, objVal = model_gs_matching(n_b, n_l, u_b, util_send_l, c, q, util_send_l, lambda_1, lambda_2, LogToConsole=False)

                for l_idx in range(1, n_l+1):
                    if l_idx in lender_matches:
                        b_match = lender_matches[l_idx] # matched borrower
                        rewards_list_l[l_idx][b_match].append(rewards(u_b[b_match][l_idx], VARIANCE)) # update the reward list for l-b pair
                        # For each lender l, the current utility for a borrower b becomes u_l(b) + [N(1, 0.25) + \sum (rewards_list)]/(1 + len(rewards_list))
                        # util_send becomes curr_util[l][b] + sqrt(2*log t/T_{b,l}(t-1))
#                         curr_util_l[l_idx][b_match] = average_reward(u_l[l_idx][b_match], rewards_list_l[l_idx][b_match])
                        util_send_l[l_idx][b_match] = reward_ucb(u_l[l_idx][b_match], rewards_list_l[l_idx][b_match], t)

                        # print(lender_matches_optimal[l_idx], b_match)
                        r = u_l[l_idx][lender_matches_optimal[l_idx]]  - u_l[l_idx][b_match]
                        # update the confidence intervals for b_match
                        lcb_m, ucb_m = cb_arms(u_l[l_idx][b_match], rewards_list_l[l_idx][b_match], t)
                        
                        # Early termination 
                        # Find the intersecting arms with b_match
                        cb_arms[l_idx][b_idx] = (lcb_m, ucb_m)
                        intersecting_arms = find_interesction_arms(lcb_m, ucb_m, cb_arms[l_idx], b_idx)
                        if len(intersecting_arms) == 0:
                            lenders_unmatched.remove(l_idx)
                        
                            #Update the borrower's remaining amount that l_idx is matched to
                            c[b_match] -= q[l_idx]
                            # TODO: Check how to run the GD ILP without one lender - maybe setting q[l_idx] = 0
                            q[l_idx] = 0
                            
                            # TODO: if borrower amount is satisfied, remove borrower from the list
                            if c[b_match] <= 0:
                                c[b_match] = 0
                                
                            
                    else:
                        r = regret_lender_t[l_idx][t-1][s_idx]
               

                    regret_lender_t[l_idx][t].append(r)

                    print("Lender {} with regret {}".format(l_idx, r))
                
              
    
    return regret_lender_t

In [ ]:
def equal_distribution_utility():
    raise NotImplementedError("Utilities not implemented")

In [29]:
def simulate_lending():
    u_b = {}
    u_l = {}
    
    n_b, n_l = 20, 60

    preference_borrowers = []
    preference_lenders = []
    
    sim_values = {}
    for b_idx in range(1, n_b+1):
        sim_values[b_idx] = {}
        for l_idx in range(1, n_l+1):
            sim_values[b_idx][l_idx]  = random.uniform(0, 1)
    
    borrower_rates = {}
    for b_idx in range(1, n_b+1):
        borrower_rates[b_idx]  = random.uniform(0, 1)

    # This part is not used for now - risk_preference
    risk_preference = {}
    categories = list(range(1, 6))
    lender_risk_categories = {}
    for l_idx in range(1, l_idx+1):
        lender_risk_categories[l_idx] = {}
        for c_idx in categories:
            lender_risk_categories[l_idx][c_idx] = random.uniform(0, 1)
    borrower_categories = {}
    for b_idx in range(1, n_b+1):
        borrower_categories[b_idx] = random.sample(categories, 1)[0] # consider every borrower has 1 category for now
        risk_preference[b_idx] = {}
        for l_idx in range(1, n_l+1):
            risk_preference[b_idx][l_idx] = lender_risk_categories[l_idx][borrower_categories[b_idx]]
    
    # c - borrower amount, q - lender amount
    c = {}
    q = {}

    while True:
        sum_c = 0
        sum_q = 0
        for b_idx in range(1, n_b+1):
            c[b_idx] = random.sample(range(5, 40), 1)[0]
            sum_c += c[b_idx]

        for l_idx in range(1, n_l+1):
            q[l_idx] = random.sample(range(1, 10), 1)[0]
            sum_q += q[l_idx]

        if sum_q > sum_c:
            break

    # utilities
    for b_idx in range(1, n_b+1):
        u_b[b_idx] = {}
        for l_idx in range(1, n_l+1):
            u_b[b_idx][l_idx] = borrower_utility(l_idx, b_idx, sim_values, risk_preference)
        preference_borrowers = sorted(range(1, len(u_b[b_idx])+1), key=lambda k: u_b[b_idx][k])

    for l_idx in range(1, n_l+1):
        u_l[l_idx] = {}
        for b_idx in range(1, n_b+1):
            u_l[l_idx][b_idx] = lender_utility(l_idx, b_idx, sim_values, q, borrower_rates)
        preference_lenders = sorted(range(1, len(u_l[l_idx])+1), key=lambda k: u_l[l_idx][k])
    
    lambda_1 = 0.5
    lambda_2 = 1-lambda_1

    print("Configuration:")
    print("Borrower preferences: ", u_b)
    print("Borrower requests: ", c)
    print("Lender prefernces: ", u_l)
    print("Lender budgets: ", q)
    
    return gs_bandit_method_basic(n_b, n_l, u_b, u_l, c, q, lambda_1, lambda_2, preference_borrowers, preference_lenders)


In [40]:
import matplotlib.pyplot as plt

def plot_lines(regret, save_dir="."):
    for l_idx in regret:
        time_points = []
        time_points_ub = []
        time_points_lb = []
        max_ub = -1
        min_lb = 10
        
        for t_idx in regret[l_idx]:
            print(t_idx)
            time_points.append(np.mean(regret[l_idx][t_idx]))
            time_points_ub.append(np.mean(regret[l_idx][t_idx]) + np.std(regret[l_idx][t_idx]))
            time_points_lb.append(np.mean(regret[l_idx][t_idx]) - np.std(regret[l_idx][t_idx]))
            
            if (np.mean(regret[l_idx][t_idx]) + np.std(regret[l_idx][t_idx])) > max_ub:
                max_ub = np.mean(regret[l_idx][t_idx]) + np.power(np.std(regret[l_idx][t_idx]), 0.5)
                                                    
            if (np.mean(regret[l_idx][t_idx]) - np.std(regret[l_idx][t_idx])) < min_lb:
                min_lb = np.mean(regret[l_idx][t_idx]) - np.power(np.std(regret[l_idx][t_idx]), 0.5)
                                                                
        x = range(len(time_points))
        plt.plot(x, time_points, 'k-')
        plt.fill_between(x, time_points_lb, time_points_ub)
        plt.ylim([max_ub+.2*max_ub, min_lb - .2*min_lb])
        plt.xlabel("Time steps", size=15)
        plt.ylabel("Regret", size=15)
        plt.title("Lender " + str(l_idx), size=20)
        plt.savefig(save_dir + "/" +"lender_img" + str(l_idx) + ".png")
        plt.close()
    

In [35]:

regret_lender = simulate_lending()



Configuration:
Borrower preferences:  {1: {1: 0.19355293507718574, 2: 0.3312892925728758, 3: 0.642069409049564, 4: 0.5332413703734583, 5: 0.33297670575028143, 6: 0.7331211811755991, 7: 0.7222091570286324, 8: 0.34694976475897266, 9: 0.4302362777035399, 10: 0.7513593639487476, 11: 0.18759648555398056, 12: 0.5405064393313554, 13: 0.5612785387499994, 14: 0.28252785358061944, 15: 0.5855808864500778, 16: 0.45942699683029403, 17: 0.43833610163557474, 18: 0.2534075598447968, 19: 0.974854259185769, 20: 0.7298410927871029, 21: 0.4836471655507204, 22: 0.5323644495047378, 23: 0.5374525596349173, 24: 0.07475354622919089, 25: 0.20412797092056445, 26: 0.4899212813381123, 27: 0.32106238988752045, 28: 0.6365974916887227, 29: 0.6099042474650658, 30: 0.32696249064217886, 31: 0.05835800986985734, 32: 0.5303876609108042, 33: 0.6106033409487269, 34: 0.704089265225909, 35: 0.49621723583781097, 36: 0.7533257033583991, 37: 0.40913161333128373, 38: 0.7969208181952963, 39: 0.42482895854321007, 40: 0.844596454174

Simulation no.: 0
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4
Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time step 9
Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching time step 14
Matching time step 15
Matching time step 16
Matching time step 17
Matching time step 18
Matching time step 19
Matching time step 20
Simulation no.: 1
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4
Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time step 9
Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching time step 14
Matching time step 15
Matching time step 16
Matching time step 17
Matching time step 18
Matching time step 19
Matching time step 20
Simulation no.: 2
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4


Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time step 9
Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching time step 14
Matching time step 15
Matching time step 16
Matching time step 17
Matching time step 18
Matching time step 19
Matching time step 20
Simulation no.: 19
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4
Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time step 9
Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching time step 14
Matching time step 15
Matching time step 16
Matching time step 17
Matching time step 18
Matching time step 19
Matching time step 20
Simulation no.: 20
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4
Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time st

Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching time step 14
Matching time step 15
Matching time step 16
Matching time step 17
Matching time step 18
Matching time step 19
Matching time step 20
Simulation no.: 37
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4
Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time step 9
Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching time step 14
Matching time step 15
Matching time step 16
Matching time step 17
Matching time step 18
Matching time step 19
Matching time step 20
Simulation no.: 38
Matching time step 1
Matching time step 2
Matching time step 3
Matching time step 4
Matching time step 5
Matching time step 6
Matching time step 7
Matching time step 8
Matching time step 9
Matching time step 10
Matching time step 11
Matching time step 12
Matching time step 13
Matching tim

In [41]:
plot_lines(regret_lender, save_dir="figure_2_21")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
1

In [13]:
for t in range(1, 20):
    print(np.sqrt(1.5*np.log(t+1)/1))

1.019666990168809
1.2837127533066595
1.442026886600883
1.5537557300461198
1.6394020872995383
1.7084686779636815
1.7661150337732119
1.815443985917585
1.8584610944249194
1.8965344471423544
1.9306371939548872
1.9614851608391803
1.989619560223232
2.0154590796275955
2.039333980337618
2.061509159835174
2.0822001913467028
2.1015847517408526
2.119810937402434
